## Importing all the Dependencies
### Music 21- A Toolkit for Computer-Aided Musical Analysis.
### glob- In Python, the glob module is used to retrieve files/pathnames matching a specified pattern. The pattern rules of glob follow standard Unix path expansion rules.
### pickle- “Pickling” is the process whereby a Python object hierarchy is converted into a byte stream, and “unpickling” is the inverse operation, whereby a byte stream (from a binary file or bytes-like object) is converted back into an object hierarchy.
### numpy - NumPy is a Python library that provides a simple yet powerful data structure
### keras - It was developed to make implementing deep learning models as fast and easy as possible for research and development.

In [1]:
from music21 import converter, instrument, note, chord, stream
import glob
import pickle
import numpy as np
from keras.utils import np_utils

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


# Preprocessing all Files
### The file is first converted into a stream.Score Object 
### Inside every single music file, every element is checked, if it's a Note or a Chord. The Note or the Chord are stored in an array called as Notes. 
### If the element is a chord, It is split into corresponding notes & concats them

In [2]:
notes = []

for file in glob.glob("Classic_Guitar/*.mid"):
    midi = converter.parse(file) # Convert file into stream.Score Object
    
    print("parsing %s"%file)
    
    elements_to_parse = midi.flat.notes
    
    
    for ele in elements_to_parse:
        # If the element is a Note,  then store it's pitch
        if isinstance(ele, note.Note):
            notes.append(str(ele.pitch))

        # If the element is a Chord, split each note of chord and join them with +
        elif isinstance(ele, chord.Chord):
            notes.append("+".join(str(n) for n in ele.normalOrder))

parsing Classic_Guitar\ss-01.mid
parsing Classic_Guitar\ss-02.mid
parsing Classic_Guitar\ss-03.mid
parsing Classic_Guitar\ss-04.mid
parsing Classic_Guitar\ss-05.mid
parsing Classic_Guitar\ss-06.mid
parsing Classic_Guitar\ss-07.mid
parsing Classic_Guitar\ss-08.mid
parsing Classic_Guitar\ss-09.mid
parsing Classic_Guitar\ss-10.mid
parsing Classic_Guitar\ss-11.mid
parsing Classic_Guitar\ss-12.mid
parsing Classic_Guitar\ss-13.mid
parsing Classic_Guitar\ss-14.mid
parsing Classic_Guitar\ss-15.mid
parsing Classic_Guitar\ss-16.mid
parsing Classic_Guitar\ss-17.mid
parsing Classic_Guitar\ss-18.mid
parsing Classic_Guitar\ss-19.mid
parsing Classic_Guitar\ss-20.mid


### Checking the Number of notes in all the files in the MIDI Music Files

In [3]:
len(notes)

9782

### We Open the Filepath to later on save the Notes in a File. 
### This Process ensures that the MIDI Files do not need to be parsed again & again to find their notes, while improving the model further in the future. 

In [4]:
with open("notes_classical_guitar", 'wb') as filepath:
    pickle.dump(notes, filepath)

### Now the Notes are stored in the file "notes_classical_guitar"

In [5]:
with open("notes_classical_guitar", 'rb') as f:
    notes= pickle.load(f)

### n_vocab stores the length of the unique notes

In [6]:
n_vocab = len(set(notes))

### The Total & Unique notes are printed

In [7]:
print("Total notes- ", len(notes))
print("Unique notes- ",  n_vocab)

Total notes-  9782
Unique notes-  181


### A Range of Notes are Printed

In [8]:
print(notes[100:200])

['F3', '7+11+2', '0+4', '2+5', 'E3', '4+7', 'F3', 'G3', '9+0+4', '2+5', '2+5+9', '0+4', 'G3', '2+5', '7+11+2', '0', 'E4', '0+2+6', 'G4', '7+11', 'D4', '10+0+4', 'F4', '5+9', 'C4', '2+5+9', 'E4', '4+8', 'B3', '7+9+1', 'D4', '2+6', 'A3', '5+7+11', '9+0+4', '2+5+9', '5+7+11', '0+4', 'E3', '0+6', 'A4', 'B3', 'G3', '2+7', '4+10', 'G4', 'F3', 'A3', '0+5', '9+2', 'F4', '4+8', '11+4', '1+7', 'E4', 'D3', 'F#3', '9+2', '5+11', 'D4', '0+4', 'C4', '0+2+6', 'D3', 'G2', 'E3', 'C4', 'B3', 'F3', 'A3', 'B3', 'F3', '0', 'F3', 'E3', 'D3', 'E3', 'E4', 'C3', 'G3', 'E3', 'G3', 'E3', 'C4', 'G3', 'E3', 'G3', 'G4', 'C3', 'G3', 'E3', 'G3', 'E3', 'E4', 'G3', 'E3', 'C4', 'G3', 'D4', 'C3']


# Prepare Sequential Data for LSTM

### We have to build a Supervised Learning Model
### Sequence Length- How many elements LSTM input should consider
### Basically, LSTM Model would take in 100 Inputs & predict one output

In [9]:
sequence_length = 100

### PitchNames stores all the Unique Notes (& Chords) in a sorted order. 

In [10]:
pitchnames = sorted(set(notes))

### Mostly ML Models do work with strings
### So, We convert the data into Integers. 
### The Following is mapping between Element (Note/Chord) to Number (Integer)
### The Mapping is stored in a Dictionary 

In [11]:
ele_to_int = dict( (ele, num) for num, ele in enumerate(pitchnames) )

### We define Network_Input & Network_Output Arrays

In [12]:
network_input = []
network_output = []

### In the following, seq_in is a set of 100 Elements (Notes/Chords) & Seq_oug is the 101th Element. This continues till all the Notes have been either a part of seq_in or seq_out
### Now, The Notes array contains strings, however, we have to work with Numerical Data, so we convert the entire data into Numericals in the following code.

In [13]:
for i in range(len(notes) - sequence_length):
    seq_in = notes[i : i+sequence_length] # contains 100 values
    seq_out = notes[i + sequence_length]
    
    network_input.append([ele_to_int[ch] for ch in seq_in])
    network_output.append(ele_to_int[seq_out])

### We are assigning n_patterns as the length of Network_Input

In [14]:
# No. of examples
n_patterns = len(network_input)
print(n_patterns)

9682


### LSTM Needs Input in 3 Dimension. 
### Now, We have Network_input & Network_Output 
### We need another Dimention, So we just take 1 as another dimention. 
### This gives us the desired shape for LSTM

In [15]:
# Desired shape for LSTM
network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
print(network_input.shape)

(9682, 100, 1)


### We Normalize the Data in the Next Step
### Normalized Data is needed for Deep Learning Models, such as this one

In [16]:
normalised_network_input = network_input/float(n_vocab)

### Network output are the classes, encode into one hot vector
### Network Output are seperated, It is turned into one vector, using Keras np_utils.to_categorical

In [17]:
network_output = np_utils.to_categorical(network_output)

### The Dimensions of Network_output can be seen in the following code

In [18]:
network_output.shape

(9682, 181)

### The Dimensions of Network_output & Normalized_network_input

In [19]:
print(normalised_network_input.shape)
print(network_output.shape)

(9682, 100, 1)
(9682, 181)


# Create Model
### Importing Necessary Libraries 
### In the previous section, Data has been made from our music, this section deals with feeding the converted data into the Music
### The Sequential model, which is very straightforward (a simple list of layers), but is limited to single-input, single-output stacks of layers (as the name gives away).
### Layers are the basic building blocks of neural networks in Keras. A layer consists of a tensor-in tensor-out computation function (the layer's call method) and some state, held in TensorFlow variables (the layer's weights).
### A callback is an object that can perform actions at various stages of training (e.g. at the start or end of an epoch, before or after a single batch, etc).

In [4]:
from keras.models import Sequential, load_model
from keras.layers import *
from keras.callbacks import ModelCheckpoint, EarlyStopping

### We have declared the model to be a sequential model. Since, It is the first later, we have to define the shape of the Input. We use the Shape of Normalized_network_input 
### Sequence has to be returned, as this is not the last layer & further layers exist.
### Dropout is a technique where randomly selected neurons are ignored during training. They are “dropped-out” randomly. This means that their contribution to the activation of downstream neurons is temporally removed on the forward pass and any weight updates are not applied to the neuron on the backward pass.
### Dropout is easily implemented by randomly selecting nodes to be dropped-out with a given probability (e.g. 20%) each weight update cycle. This is how Dropout is implemented in Keras. Dropout is only used during the training of a model and is not used when evaluating the skill of the model.
### Application of dropout at each layer of the network has shown good results.
### Another LSTM Layer is added with 512 Units & Return Sequence is true, as this is another layer which is not the output layer. 
### In the last Layer, Again the Units 512 & Return sequence is false by default. 
### At the end, there's a dense layer of 256 Layers & a dropout of 0.3. 
### In the final layer, The Dense layer should have units equal to n_vocab. it should have it's activation function as softmax.
### Softmax is typically used as the activation function when 2 or more class labels are present in the class membership in the classification of multi-class problems. It is also a general case of the sigmoid function when it represents the binary variable probability distribution.

In [21]:
model = Sequential()
model.add( LSTM(units=512,
               input_shape = (normalised_network_input.shape[1], normalised_network_input.shape[2]),
               return_sequences = True) )
model.add( Dropout(0.3) )
model.add( LSTM(512, return_sequences=True) )
model.add( Dropout(0.3) )
model.add( LSTM(512) )
model.add( Dense(256) )
model.add( Dropout(0.3) )
model.add( Dense(n_vocab, activation="softmax") )

### Now, The Model is compiled , using the Adam Optimizer 
### The purpose of loss functions is to compute the quantity that a model should seek to minimize during training.
### The Model tries to minimize the Categorical crossentropy loss value.
### Adam is a replacement optimization algorithm for stochastic gradient descent for training deep learning models. Adam combines the best properties of the AdaGrad and RMSProp algorithms to provide an optimization algorithm that can handle sparse gradients on noisy problems.

In [22]:
model.compile(loss="categorical_crossentropy", optimizer="adam")

### A Summary of the Model is shown in the following code block

In [23]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 100, 512)          1052672   
_________________________________________________________________
dropout (Dropout)            (None, 100, 512)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100, 512)          2099200   
_________________________________________________________________
dropout_1 (Dropout)          (None, 100, 512)          0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 512)               2099200   
_________________________________________________________________
dense (Dense)                (None, 256)               131328    
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0

### The following code block stores the Model in the file "model_classical_guitar.hdf5", while the loss is monitored. The Mode is Minimum, as the loss is to be minimized. 
### The statement below the same (model.fit) is used to start the actual training. 
### The Various parameters like epochs define the number of interations of the training & the other parameters which will be used to train the model.

In [24]:
checkpoint = ModelCheckpoint("model_classical_guitar.hdf5", monitor='loss', verbose=0, save_best_only=True, mode='min')


model_his = model.fit(normalised_network_input, network_output, epochs=10, batch_size=64, callbacks=[checkpoint])

Epoch 1/10
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: invalid syntax (tmpu5n9gzu1.py, line 48)
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: invalid syntax (tmpu5n9gzu1.py, line 48)
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
  4/152 [..............................] - ETA: 14:37 - loss: 5.1073

KeyboardInterrupt: 

### Now, We load the model to start making new music. 
### The Training has been done through Google Colab, to ensure that the Model Trains quickly.

In [5]:
model = load_model("model_classical_guitar.hdf5")

# Predictions

### Network_Input is now an np array, It is thus declared again
### Network_Input here is basically a list of list 

In [6]:
sequence_length = 100
network_input = []

for i in range(len(notes) - sequence_length):
    seq_in = notes[i : i+sequence_length] # contains 100 values
    network_input.append([ele_to_int[ch] for ch in seq_in])

NameError: name 'notes' is not defined

### The Start is initialized with a random number, to generate different music every time 
### Integer to Element Mapping is created, as the Element to Integer Map was declared above. A dictionary is used again
### Initial Pattern the starting index from where, the 100 Notes(or Chords) are taken
### 200 Elements are generated, which should produce music for around 1 min. 
### In the for loop, initially 100 elements are taken from the start point, then prediction is made using LSTM, Now, the 1st Element is removed & the 1-101 elements are used to carry out the same process. 
### Prediction input is used to get the data into the standard format, suitable for the model. 
### Prediction gives the probability of each unique element. The Element( Note/Chord) which has the Max Probability is taken. 
### The Result is then appended to the output & the pattern is changed to include the next 100 elements, not considering the first element. The recent value predicted by the Model is Appended next. 

In [62]:
# Any random start index
start = np.random.randint(len(network_input) - 1)

# Mapping int_to_ele
int_to_ele = dict((num, ele) for num, ele in enumerate(pitchnames))

# Initial pattern 
pattern = network_input[start]
prediction_output = []

# generate 200 elements
for note_index in range(300):
    prediction_input = np.reshape(pattern, (1, len(pattern), 1)) # convert into numpy desired shape 
    prediction_input = prediction_input/float(n_vocab) # normalise
    
    prediction =  model.predict(prediction_input, verbose=0)
    
    idx = np.argmax(prediction)
    result = int_to_ele[idx]
    prediction_output.append(result) 
    
    # Remove the first value, and append the recent value.. 
    # This way input is moving forward step-by-step with time..
    pattern.append(idx)
    pattern = pattern[1:]

### The Output Elements (Notes/Chords) can be seen here

In [63]:
print(prediction_output)

['E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'A3', 'E4', 'E4', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4

# Create Midi File

### The following code section basically iterates over the Output Elements & Checks whether they are Notes or chords. 
### Output_notes is used to store the the output Notes or Chords
## If the Element is a note, 
### A note object is made, using the class note in the note subpackage. 
### If Note is found, It is given an Offset & Instrument is set from a list of Instruments in the Music 21 Library
### Output is appended with this New Note along with it's offset. 
## Else, if it's Chord 
### If + is a part of the patter, or the patter is a complete digit, then the pattern is a chord. 
### Every single Note is seperated out by checking where the '+' sign is. Thus, getting all the notes of the chord. 
### The array temp_notes is used to store the data of every single note & then further the list of notes (temp_notes) are converted into the corresponding chord object as new_chord
### The Note is then appended into the output_notes array
### Offset says, that after the first note, the second note should start playing after 0.5 

In [64]:
import random
offset = 0 # Time
output_notes = []

for pattern in prediction_output:
    
    # if the pattern is a chord
    if ('+' in pattern) or pattern.isdigit():
        notes_in_chord = pattern.split('+')
        temp_notes = []
        for current_note in notes_in_chord:
            new_note = note.Note(int(current_note))  # create Note object for each note in the chord
            new_note.storedInstrument = instrument.Piano()
            temp_notes.append(new_note)
            
        
        new_chord = chord.Chord(temp_notes) # creates the chord() from the list of notes
        new_chord.offset = offset
        output_notes.append(new_chord)
    
    else:
            # if the pattern is a note
        new_note = note.Note(pattern)
        new_note.offset = offset
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note)
        
    offset = 0.5

## Creating a Stream Object from the generated Notes 
## Writing the Stream Object into a test_output file. 

In [65]:
# create a stream object from the generated notes
midi_stream = stream.Stream(output_notes)
midi_stream.write('midi', fp = "test_output.mid")

'test_output.mid'

In [66]:
midi_stream.show('midi')